In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.neural_network import MLPClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [3]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [5]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [7]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

# Pre-processing

In [9]:
# not necessary

# Exploratory data analysis

In [10]:
X_train.isnull().sum()

1      0
2      0
3      0
4      0
5      0
      ..
442    0
443    0
444    0
445    0
446    0
Length: 446, dtype: int64

# Model selection

In [11]:
# Grid Search

cv = 5         # number of folds
verbose = 3     # information shown during training

## Multi-layer Perceptron

In [13]:
parameters = {
    "hidden_layer_sizes":[(100,), (200,), (300,)],
    "alpha":[0.001, 0.01]}
model = GridSearchCV(MLPClassifier(), parameters, cv=cv, verbose=verbose, scoring="f1_weighted")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_hidden_layer_sizes", "param_alpha", "mean_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END alpha=0.001, hidden_layer_sizes=(100,);, score=0.690 total time=   6.8s
[CV 2/5] END alpha=0.001, hidden_layer_sizes=(100,);, score=0.693 total time=   7.1s
[CV 3/5] END alpha=0.001, hidden_layer_sizes=(100,);, score=0.683 total time=   6.9s
[CV 4/5] END alpha=0.001, hidden_layer_sizes=(100,);, score=0.664 total time=   6.8s
[CV 5/5] END alpha=0.001, hidden_layer_sizes=(100,);, score=0.661 total time=   6.8s
[CV 1/5] END alpha=0.001, hidden_layer_sizes=(200,);, score=0.693 total time=   7.2s
[CV 2/5] END alpha=0.001, hidden_layer_sizes=(200,);, score=0.691 total time=   7.4s
[CV 3/5] END alpha=0.001, hidden_layer_sizes=(200,);, score=0.667 total time=   7.2s
[CV 4/5] END alpha=0.001, hidden_layer_sizes=(200,);, score=0.672 total time=   7.2s
[CV 5/5] END alpha=0.001, hidden_layer_sizes=(200,);, score=0.648 total time=   6.9s
[CV 1/5] END alpha=0.001, hidden_layer_sizes=(300,);, score=0.691 total time=  12.1s
[CV 2

,param_hidden_layer_sizes,param_alpha,mean_test_score
5,"(300,)",0.01,0.689202
4,"(200,)",0.01,0.681054
3,"(100,)",0.01,0.679244
0,"(100,)",0.001,0.678205
1,"(200,)",0.001,0.674371
2,"(300,)",0.001,0.673890


# Final model

In [ ]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

In [ ]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [ ]:
predictions.to_csv("submission.csv", index=False)